In [24]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.python.framework.ops import EagerTensor
import pprint
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
%matplotlib inline

In [25]:
IMAGE_WIDTH=64
IMAGE_HEIGHT=64
IMAGE_SIZE=(IMAGE_WIDTH,IMAGE_HEIGHT)
IMAGE_CHANNEL=3
IMAGE_PATH = './pet_images'
BATCH_SIZE = 20
EPOCHS = 64
CLASSES = os.listdir(IMAGE_PATH)
NUM_CLASSES = len(CLASSES)

In [26]:
cat_image_dirs=os.listdir("./pet_images/Cat")
dog_image_dirs=os.listdir("./pet_images/Dog")
image_dirs = cat_image_dirs + dog_image_dirs

labels = list(map(lambda x: 'cat', cat_image_dirs)) + list(map(lambda x: 'dog', dog_image_dirs))
len(labels)

24961

In [27]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.2)

train_dataset = train_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size = BATCH_SIZE,
    subset = "training",
    class_mode = "categorical",
    shuffle = True
)

val_dataset = train_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size = BATCH_SIZE,
    subset = "validation",
    class_mode = "categorical",
    shuffle = True
)

Found 19968 images belonging to 2 classes.
Found 4991 images belonging to 2 classes.


In [28]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ### START CODE HERE
    ## Second component of main path (≈3 lines)
    ## Set the padding = 'same'
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    ## Set the padding = 'valid'
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    ### END CODE HERE
    return X

In [29]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ### START CODE HERE
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [30]:
def ResNet50(input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3), classes = 2):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [IMAGE_HEIGHT, IMAGE_WIDTH, 256], s = 1)
    X = identity_block(X, 3, [IMAGE_HEIGHT, IMAGE_WIDTH, 256])
    X = identity_block(X, 3, [IMAGE_HEIGHT, IMAGE_WIDTH, 256])

    ### START CODE HERE
    
    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [IMAGE_HEIGHT * 2, IMAGE_WIDTH * 2,512], s = 2)
    X = identity_block(X, 3, [IMAGE_HEIGHT * 2, IMAGE_WIDTH * 2,512])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 2, IMAGE_WIDTH * 2,512])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 2, IMAGE_WIDTH * 2,512])
    
    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [IMAGE_HEIGHT * 4, IMAGE_WIDTH * 4, 1024], s = 2)
    X = identity_block(X, 3, [IMAGE_HEIGHT * 4, IMAGE_WIDTH * 4, 1024])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 4, IMAGE_WIDTH * 4, 1024])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 4, IMAGE_WIDTH * 4, 1024])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 4, IMAGE_WIDTH * 4, 1024])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 4, IMAGE_WIDTH * 4, 1024])

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [IMAGE_HEIGHT * 8, IMAGE_WIDTH * 8, 2048], s = 2)
    X = identity_block(X, 3, [IMAGE_HEIGHT * 8, IMAGE_WIDTH * 8, 2048])
    X = identity_block(X, 3, [IMAGE_HEIGHT * 8, IMAGE_WIDTH * 8, 2048])

    ## AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2))(X)
    
    ### END CODE HERE

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [31]:
model = ResNet50(input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3), classes = NUM_CLASSES)
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_3 (ZeroPadding2  (None, 70, 70, 3)   0           ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_159 (Conv2D)            (None, 32, 32, 64)   9472        ['zero_padding2d_3[0][0]']       
                                                                                                  
 batch_normalization_159 (Batch  (None, 32, 32, 64)  256         ['conv2d_159[0][0]']       

                                                                                                  
 conv2d_169 (Conv2D)            (None, 15, 15, 256)  16640       ['activation_155[0][0]']         
                                                                                                  
 batch_normalization_169 (Batch  (None, 15, 15, 256)  1024       ['conv2d_169[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_50 (Add)                   (None, 15, 15, 256)  0           ['batch_normalization_169[0][0]',
                                                                  'activation_153[0][0]']         
                                                                                                  
 activation_156 (Activation)    (None, 15, 15, 256)  0           ['add_50[0][0]']                 
          

 add_53 (Add)                   (None, 8, 8, 512)    0           ['batch_normalization_179[0][0]',
                                                                  'activation_162[0][0]']         
                                                                                                  
 activation_165 (Activation)    (None, 8, 8, 512)    0           ['add_53[0][0]']                 
                                                                                                  
 conv2d_180 (Conv2D)            (None, 8, 8, 128)    65664       ['activation_165[0][0]']         
                                                                                                  
 batch_normalization_180 (Batch  (None, 8, 8, 128)   512         ['conv2d_180[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

                                                                                                  
 batch_normalization_190 (Batch  (None, 4, 4, 256)   1024        ['conv2d_190[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 4, 4, 256)    0           ['batch_normalization_190[0][0]']
                                                                                                  
 conv2d_191 (Conv2D)            (None, 4, 4, 256)    590080      ['activation_175[0][0]']         
                                                                                                  
 batch_normalization_191 (Batch  (None, 4, 4, 256)   1024        ['conv2d_191[0][0]']             
 Normalization)                                                                                   
          

 activation_185 (Activation)    (None, 4, 4, 256)    0           ['batch_normalization_200[0][0]']
                                                                                                  
 conv2d_201 (Conv2D)            (None, 4, 4, 1024)   263168      ['activation_185[0][0]']         
                                                                                                  
 batch_normalization_201 (Batch  (None, 4, 4, 1024)  4096        ['conv2d_201[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_60 (Add)                   (None, 4, 4, 1024)   0           ['batch_normalization_201[0][0]',
                                                                  'activation_183[0][0]']         
                                                                                                  
 activatio

                                                                                                  
 add_63 (Add)                   (None, 2, 2, 2048)   0           ['batch_normalization_211[0][0]',
                                                                  'activation_192[0][0]']         
                                                                                                  
 activation_195 (Activation)    (None, 2, 2, 2048)   0           ['add_63[0][0]']                 
                                                                                                  
 average_pooling2d_3 (AveragePo  (None, 1, 1, 2048)  0           ['activation_195[0][0]']         
 oling2D)                                                                                         
                                                                                                  
 flatten_3 (Flatten)            (None, 2048)         0           ['average_pooling2d_3[0][0]']    
          

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, validation_data = val_dataset, epochs = EPOCHS, batch_size = BATCH_SIZE)

Epoch 1/64
999/999 [==============================] - 1338s 1s/step - loss: 0.8384 - accuracy: 0.5997 - val_loss: 0.7205 - val_accuracy: 0.5444
Epoch 2/64
999/999 [==============================] - 1291s 1s/step - loss: 0.6938 - accuracy: 0.6022 - val_loss: 0.6664 - val_accuracy: 0.6147
Epoch 3/64
999/999 [==============================] - ETA: 0s - loss: 0.6405 - accuracy: 0.6548